In [1]:
import os, sys
import pandas as pd
import numpy as np
import torch
import torch.optim as optim

from torch.utils.data import DataLoader

In [2]:
sys.path.append('../src')
from data.dataset import GWDataset, Normalize
from model.handler import ModelHandler
from model.mlp import MLP

In [3]:
base_data_dir = '/srv/scratch/z5370003/projects/data'
data_dir = os.path.join(base_data_dir, 
                        'groundwater/FEFLOW/coastal/',
                         'variable_density/')
data_path = os.path.join(data_dir, 'all')

In [4]:
_mean = np.array([1523.10199582, 1729.62046887,   26.72577967,  316.5513978])
_std = np.array([569.13566635, 566.33636362,  14.90088159, 183.2160048 ])
input_transform = Normalize(mean=_mean, std=_std)

In [5]:
_mean = np.array([3.29223762e-01, 1.78798090e+04])
_std = np.array([2.17284490e-01, 1.51990336e+04])
output_transform = Normalize(mean=_mean, std=_std)

In [6]:
test_ds = GWDataset(data_path, 'val', 
                     input_transform=input_transform, 
                     output_transform=output_transform,
                     val_ratio=0.3)


test_dl = DataLoader(test_ds, batch_size=10240, 
                      shuffle=False, pin_memory=True)
len(test_dl)

3434

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
input_size = 4
hidden_sizes = [32, 64, 128, 32, 128, 32]
output_size = 2

mlp = MLP(input_size=input_size,
          hidden_sizes=hidden_sizes,
          output_size=output_size).double()

In [11]:
model_file = '../logs/saves/mlp_model'
mlp.load_state_dict(torch.load(model_file))
mlp.eval()

/scratch/pbs.6561716.kman.restech.unsw.edu.au/ipykernel_3925662/2428229336.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mlp.load_state_dict(torch.load(model_file))


MLP(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=128, bias=True)
    (9): ReLU()
    (10): Linear(in_features=128, out_features=32, bias=True)
    (11): ReLU()
    (12): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [12]:
model_handler = ModelHandler(model=mlp, device=device)

In [ ]:
preds = model_handler.predict(test_dl)

  0%|          | 7/3434 [00:18<2:31:27,  2.65s/it]

In [ ]:
# print(f"Train loss: {model_handler.evaluate(train_dl)}")
# print(f"Test loss: {model_handler.evaluate(test_dl)}")

In [ ]:
from tqdm import tqdm

inputs = []
targets = []

for X, y in tqdm(test_dl):
    inputs.append(X.detach().numpy())
    targets.append(y.detach().numpy())

In [ ]:
write_dir = '/srv/scratch/z5370003/projects/04_groundwater/variable_density/results/MLP/'
np.save(os.path.join(write_dir, 'preds.npy'), preds)
np.save(os.path.join(write_dir, 'inputs.npy'), np.concat(inputs))
np.save(os.path.join(write_dir, 'targets.npy'), np.concat(targets))